# Elliptic Curves
In this document I am going to play around with some implementations and computations involving elliptic curves.

---
As a guide I referred to [this](https://jeremykun.com/2014/02/24/elliptic-curves-as-python-objects/) guide to ellpitic curve operations in Python as well as [this](https://www.nayuki.io/page/elliptic-curve-point-addition-in-projective-coordinates) discussion and implementation of elliptic curves in projective coordinates.

In [51]:
class EllipticCurve:
    """
    A class that represents an elliptic curve in P^2 in the form
    y^2z = x^3 + axz^2 + bz^3 over a field k.
    """
    def __init__(self, a, b, k):
        # Check to make sure inputs make sense.
        if not k.real and isinstance(a, ComplexNumber) and isinstance(b, ComplexNumber):
            pass
        elif k.char > 0 and isinstance(a, int) and isinstance(b, int):
            pass
        elif k.char == 0:
            pass
        else:
            raise ValueError('Field must contain a and b.')
        
        self.a = a
        self.b = b
        self.field = k
        self.modulus = k.char
        
        # Make sure the curve is not singular via the discriminant
        if abs(a) > 0:
            pass
        elif abs(4*a**3 + 27*b**2) == 0:
            raise ValueError('The curve is singular!')
        
    def __str__(self):
        return "Elliptic curve y^2z = x^3 + %sxz^2 + %sz^3 over %s." % (self.a, self.b, self.field)
    
    def __eq__(self, other):
        # In general it is difficult to determine whether two curves are the same. For us, 
        # we just want to check whether they were instantiated with the same parameters.
        return (self.a, self.b, self.field) == (other.a, other.b, other.field)
    
    def containsPoint(self, x,y,z):
        lhs = y*y*z
        rhs = x*x*x + self.a * x*z*z + self.b * z*z*z
        
        if self.modulus > 0:
            lhs = lhs % self.modulus
            rhs = rhs % self.modulus
            
        return lhs == rhs

## Fields and Points
We want our curves to be able to be defined over different fields, so we should define some notion of them to include in the definition of each curve. 

We also define points in $\mathbb{P}^2$ over any field.

In [52]:
import math

class Field:
    """
    Represents a field. For now, options are characteristic zero (R or C) and finite fields.
    """
    def __init__(self, characteristic=0, cardinality=math.inf, real=True):
        # Make sure we're all sane here
        if not isinstance(characteristic, int) or characteristic < 0:
            raise TypeError("Characteristic must be 0 or a positive integer (prime).")
        if not isinstance(cardinality, int) and cardinality != math.inf:
            raise TypeError("Cardinality must be integral or infinite.")
        if not isinstance(real, bool):
            raise TypeError("Variable 'real' must be a boolean!")
        
        # Store our class variables.
        self.char = characteristic
        self.card = cardinality
        self.real = real
        
        # R and C must be infinite cardinality!
        if self.char == 0 and self.card < math.inf:
            raise ValueError('Fields of characteristic zero must be infinite.')
        # Finite fields must be the right order
        if self.card < math.inf:
            dim = int(math.log(self.card)/math.log(self.char))
            if self.char**dim != self.card:
                raise ValueError('Finite fields of characteristic p must be of size p^k.')
        # No infinite fields of characteristic p
        if self.char > 0 and self.card == math.inf:
            raise ValueError('No infinite fields with positive characteristic.')
        
    def __str__(self):
        if not self.real:
            return 'the complex numbers'
        elif self.char == 0:
            return 'the real numbers'
        else:
            return 'the finite field F_(%d^%d)' % (self.char,int(math.log(self.card)/math.log(self.char)))
        
    def __eq__(self, other):
        return (self.char, self.card, self.real) == (other.char, other.card, other.real)

R = Field()
C = Field(real=False)
F8 = Field(characteristic=2, cardinality=8)
G = Field(characteristic=2, cardinality=8)

curve = EllipticCurve(0,17,R)
curve2 = EllipticCurve(1,2,G)
curve3 = EllipticCurve(ComplexNumber(1),ComplexNumber(2),C)

In [53]:
#testing

print(R)
print(C)
print(F8)

print(curve3)
print(curve.containsPoint(5,2,5))
print(R == C)
print(F8 == G)
print(curve == curve2)
print(curve == curve3)

the real numbers
the complex numbers
the finite field F_(2^3)
Elliptic curve y^2z = x^3 + 1xz^2 + 2z^3 over the complex numbers.
False
False
True
False
False


In [54]:
class Point:
    """
    Represents a (projective) point on a curve.
    """
    def __init__(self,x,y,z,curve):
        if (x,y,z) == (0,0,0):
            raise ValueError("Projective coordinates can't all be zero!")
        if not curve.containsPoint(x,y,z):
            raise ValueError("Point does not lie on the given curve: [%s:%s:%s]" % (x,y,z))
        self.modulus = curve.modulus
        self.field = curve.field
        self.x = x
        self.y = y
        self.z = z
        self.curve = curve
    
    """
    Addition here is going to be addition on the elliptic curve.
    """
    def __add__(self, other):
        # First verify that you can actually add these points.
        if self.curve == other.curve and self.curve.containsPoint(other.x,other.y,other.z):
            pass
        else:
            raise ValueError('Attempted to add points from different curves!')
            
        if self.is_identity():
            return other
        elif other.is_identity():
            return self
        elif self == -other or self.y == 0:
            return Point(0,1,0,curve)
        elif self.z == 0:
            # Should never get here although it's only implied.
            raise ValueError('Something is wrong.')
        elif self == other:
            # I spent so much time getting rid of the divisions here and I am not sure it was worth it. XD
            x = self.x/self.z
            y = self.y/self.z
            s = (3*x**2+curve.a)/(2*y)
            newX = s**2-2*x
            newY = s*(newX - x)+y
            return Point(newX, newY, 1, curve)
        else:
            s = (self.y-other.y)/(self.x-other.x)
            newX = s**2 - self.x - other.x
            newY = s*(newX - self.x) + self.y
            return Point(newX, newY, 1, curve)
    
    def __neg__(self):
        return Point(self.x, -self.y, self.z, curve)
    
    def __str__(self):
        if self.is_identity():
            return "[0:1:0]"
        elif self.z == 0:
            return "[1:%s:0]" % (self.y/self.x)
        else:
            return "[%s:%s:1]" % (self.x/self.z, self.y/self.z)
    
    def __sub__(self, other):
        return self.__add__(-other)
    
    def __eq__(self, other):
        if self.z != 0 and other.z != 0:
            return (self.x/self.z, self.y/self.z) == (other.x/other.z, other.y/other.z)
        elif self.y != 0 and other.y != 0:
            return (self.x/self.y, self.z/self.y) == (other.x/other.y, other.z/other.y)
        else:
            return (self.z/self.x, self.y/self.x) == (other.z/other.x, other.y/other.x)
    
    def is_identity(self):
        return self.x == 0 and self.z == 0
    
    """
    Multiplication here is shorthand for iterated self-addition.
    
    """
    def __rmul__(self, coef):
        return self*coef
    
    def __mul__(self, coef):
        # First verify that you can actually add these points.
        if isinstance(coef, int):
            pass
        else:
            raise ValueError('Cannot multiply projective points by non-integers!')
        
        # Compute using a recursive definition
        if coef < 0:
            return -((-coef)*self)
        elif coef == 0:
            return Point(0,1,0,curve)
        elif coef == 1:
            return self
        else:
            return self + (coef - 1)*self

In [56]:
P = Point(-1,4,1,curve)
print(P)
Q = Point(2,5,1,curve)
print(2*P)
print(P+Q)

[-1.0:4.0:1]
[2.140625:5.177734375:1]
[-0.8888888888888888:4.037037037037037:1]


In [57]:
%timeit P+Q

6.17 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Complex Numbers
I really wanted to implement my own version of $\mathbb{C}$ so I did. It ends up that the native version of the complex numbers runs about 16 times faster for most operations. Not terribly surprising since it's probably implemented at a much lower level. I don't think you can make these computations more efficient (besides maybe the exponentiation).

In [5]:
import math

class ComplexNumber:
    """
    Represents a complex number with real and imaginary parts as well as the operations that accompany them.
    
    Args:
        real: The real part of the number
        imaginary: The imaginary part of the number
    """
    def __init__(self, real, imaginary=0):
        self.real = real
        self.imag = imaginary
    
    def __str__(self):
        if self.imag == 0:
            return str(self.real)
        elif self.real == 0:
            return "%si" % self.imag
        elif self.imag > 0:
            return "%s + %si" % (self.real, self.imag)
        else:
            return "%s - %si" % (self.real, -self.imag)
    
    def __add__(self, other):
        return ComplexNumber(self.real+other.real, self.imag+other.imag)
    
    def __neg__(self):
        return ComplexNumber(-self.real, -self.imag)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __abs__(self):
        return math.sqrt(self.real**2 + self.imag**2)
    
    def __mul__(self, other):
        if isinstance(other, ComplexNumber):
            return ComplexNumber(self.real*other.real-self.imag*other.imag, self.real*other.imag+self.imag*other.real)
        else:
            return ComplexNumber(other*self.real, other*self.imag)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __truediv__(self, other):
        if other.is_zero():
            raise ZeroDivisionError()
        newX = self.real*other.real + self.imag*other.imag
        newY = self.imag*other.real - self.real*other.imag
        factor = other.real**2+other.imag**2
        return ComplexNumber(newX/factor, newY/factor)
    
    def __pow__(self, exp):
        mag = abs(self)**exp
        angle = math.atan(self.imag/self.real)
        if self.real < 0:
            angle += math.pi
        angle *= exp
        return mag*ComplexNumber(math.cos(angle), math.sin(angle))
    
    def is_zero(self):
        return self.real == 0 and self.imag == 0
    
    def conjugate(self):
        return ComplexNumber(self.real, -self.imag)

z = ComplexNumber(1,2)
w = ComplexNumber(4,2)

a = complex(1,2)
b = complex(4,2)

In [99]:
%timeit z**12.3
%timeit a**12.3

2.85 µs ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
224 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [36]:
print((z**12.3))
print((a**12.3).real)
print(math.pi*z)
print(w-z)

9870.483984788209 + 17269.674586771587i
9870.483984788209
3.141592653589793 + 6.283185307179586i
3


In [114]:
print(z*z.conjugate())
print(a.conjugate())

5
(1-2j)


In [80]:
1000/60

16.666666666666668